In [5]:
import os
from keras import backend as K
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Força o uso da CPU

In [6]:
from Modelos import GeradorClassificador, Gerador
from segmentandoDatasets import segmentando_datasets
from Preprocessamento import preprocessamento

In [ ]:
segmentando_datasets()

Criação dos Modelos a partir de um gerador, todos os modelos estão com a entrada de imagem (64,64,3)

In [ ]:
treinoPUC, validacaoPUC, testePUC, treino_dfPUC, validacao_dfPUC, teste_df = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_PUC.csv")
PUC = Gerador(input_shape=(64,64,3), max_layers=4)
PUC.Dataset(treinoPUC, validacaoPUC, testePUC)

ModeloPUC = PUC.construir_modelo()
PUC.compilar_modelo()
ModeloPUC.summary()
PUC.encoder.summary()
PUC.decoder.summary()
PUC.setNome("Modelo PUC - Teste")

PUC.treinar_autoencoder(epocas=10, salvar=True, batch_size=8)
del ModeloPUC

K.clear_session()


In [ ]:
treinoUFPR04, validacaoUFPR04, testeUFPR04, treino_dfUFPR04, validacao_dfUFPR04, teste_dfUFPR04 = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_UFPR04.csv")
UFPR04 = Gerador(input_shape=(64,64,3), max_layers=4)
UFPR04.Dataset(treinoUFPR04, validacaoUFPR04, testeUFPR04)

ModeloUFPR04 = UFPR04.construir_modelo()
UFPR04.compilar_modelo()
ModeloUFPR04.summary()
UFPR04.encoder.summary()
UFPR04.decoder.summary()
UFPR04.setNome("Modelo UFPR04")

UFPR04.treinar_autoencoder(epocas=10, salvar=True, batch_size=8)
del ModeloUFPR04

K.clear_session()


In [ ]:
treinoUFPR05, validacaoUFPR05, testeUFPR05, treino_dfUFPR05, validacao_dfUFPR05, teste_dfUFPR05 = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_UFPR05.csv")
UFPR05 = Gerador(input_shape=(64,64,3), max_layers=4) 
UFPR05.Dataset(treinoUFPR05, validacaoUFPR05, testeUFPR05)

ModeloUFPR05 = UFPR05.construir_modelo()
UFPR05.compilar_modelo()
ModeloUFPR05.summary()
UFPR05.encoder.summary()
UFPR05.decoder.summary()

UFPR05.setNome("Modelo UFPR05")

UFPR05.treinar_autoencoder(epocas=10, salvar=True, batch_size=8)
del ModeloUFPR05

K.clear_session()


Forma como posso carregar os modelos

In [8]:
PUC = Gerador()
UFPR04 = Gerador()
UFPR05 = Gerador()

PUC.setNome('Modelo PUC')
UFPR04.setNome('Modelo UFPR04')
UFPR05.setNome('Modelo UFPR05')

AutoencoderPUC = PUC.carrega_modelo('Modelos_keras/Autoencoders_Gerados/Modelo PUC.keras', 'weights_finais/Autoencoders_Gerados/Modelo PUC.weights.h5')
AutoencoderUFPR04 = UFPR04.carrega_modelo('Modelos_keras/Autoencoders_Gerados/Modelo UFPR04.keras', 'weights_finais/Autoencoders_Gerados/Modelo UFPR04.weights.h5')
AutoencoderUFPR05 = UFPR05.carrega_modelo('Modelos_keras/Autoencoders_Gerados/Modelo UFPR05.keras', 'weights_finais/Autoencoders_Gerados/Modelo UFPR05.weights.h5')

In [ ]:
segmentando_datasets(10000,10000,10000)

Gero meus classificadores a partir dos Encoders

In [ ]:
classificadorPUC = GeradorClassificador(PUC.encoder, PUC.getPesos())
treinoPUC, validacaoPUC, testePUC, treino_dfPUC, validacao_dfPUC, teste_dfPUC = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_PUC.csv", autoencoder=False)
classificadorPUC.Dataset(treinoPUC, validacaoPUC, testePUC)
classificadorPUC.treinamento(epocas=5)
classificadorPUC.predicao(teste_dfPUC)

In [ ]:
classificadorUFPR04 = GeradorClassificador(UFPR04.encoder, UFPR04.getPesos())
treinoUFPR04, validacaoUFPR04, testeUFPR04, treino_dfUFPR04, validacao_dfUFPR04, teste_dfUFPR04 = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_UFPR04.csv", autoencoder=False)
classificadorUFPR04.Dataset(treinoUFPR04, validacaoUFPR04, testeUFPR04)
classificadorUFPR04.treinamento()
classificadorUFPR04.predicao(teste_dfUFPR04)

In [ ]:
classificadorUFPR05 = GeradorClassificador(UFPR05.encoder, UFPR05.getPesos())
treinoUFPR05, validacaoUFPR05, testeUFPR05, treino_dfUFPR05, validacao_dfUFPR05, teste_dfUFPR05 = preprocessamento(input_shape = (64,64), caminho="Datasets_csv/df_UFPR05.csv", autoencoder=False)
classificadorUFPR05.Dataset(treinoUFPR05, validacaoUFPR05, testeUFPR05)
classificadorUFPR05.treinamento()
classificadorUFPR05.predicao(teste_dfUFPR05)

Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X = teste_dfPUC['caminho_imagem']
y = teste_dfPUC['classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ensemble = VotingClassifier(
    estimators=[
        ('PUC', classificadorPUC),
        ('UFPR04', classificadorUFPR04),
        ('UFPR05', classificadorUFPR05)
    ],
    voting='hard'
)
ensemble.fit(X_train, y_train)

print(f"Accuracy of the ensemble: {ensemble.score()*100} %")

Predizendo a mesma base para todos e criar um df para comparar seus resultados

In [ ]:
predPuc = classificadorPUC.predicao(teste_dfUFPR04)
predUfpr04 = classificadorUFPR04.predicao(teste_dfUFPR04)
predUfpr05 = classificadorUFPR05.predicao(teste_dfUFPR04)

In [ ]:
import pandas as pd
from Preprocessamento import mapear_rotulos_binarios
from visualizacao import exibir_primeiras_imagens

comparacao = pd.DataFrame()
comparacao['Classe real'] = mapear_rotulos_binarios(teste_dfUFPR04['classe'])
comparacao['Classe predida PUC'] = predPuc
comparacao['Classe predida UFPR04'] = predUfpr04
comparacao['Classe predida UFPR05'] = predUfpr05

comparacao.to_csv("teste.csv", index=False)

exibir_primeiras_imagens(dataframe=teste_dfUFPR04)
